In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from matplotlib import pyplot as plt
%matplotlib inline

import torch
import numpy as np

np.set_printoptions(precision=2, floatmode='fixed')

import tsvar
from gb import GrangerBusca, simulate

In [4]:
print('  - Define model parameters...')
# Define random parameters
dim = 2  # Dimensionality of the process
end_time = 3e5  # Choose a long observation window
mu = torch.tensor([0.4, 0.1])
beta = torch.tensor([  # Must still be beta_j^i equal for all i in simulator
    [1.1, 1.0],
    [1.2, 1.2]
])
alpha = torch.tensor([
    [0.7, 0.3],
    [0.0, 1.0]
])
coeffs_true = torch.cat((mu, beta.flatten(), alpha.flatten())).numpy()
print('  - Simulate lots of data...')
# Simulate lots of data
wold_sim = simulate.GrangeBuscaSimulator(
    mu_rates=mu, Alpha_ba=alpha, Beta_ba=beta)
events = wold_sim.simulate(end_time)
events = [torch.tensor(ev, dtype=torch.float) for ev in events]
print((f"    - Simulated {sum(map(len, events)):,d} events "
       f"with end time: {end_time}"))


  - Define model parameters...
  - Simulate lots of data...
    - Simulated 417,296 events with end time: 300000.0


In [5]:
print('  - Run Granger Busca...')
# Define model
granger_model = GrangerBusca(
    alpha_prior=1.0/len(events), 
    num_iter=3000, 
    metropolis=True, 
    beta_strategy='busca',
) #recommended parameters
granger_model.fit(events)

adj_hat = granger_model.Alpha_.toarray()
adj_hat = adj_hat / adj_hat.sum(axis=0)

beta_hat = np.ones((dim, dim)) * granger_model.beta_ + 1

coeffs_hat = np.hstack((granger_model.mu_, beta_hat.flatten(), adj_hat.flatten()))

max_diff = np.max(np.abs(coeffs_true - coeffs_hat))
print(f'  - coeffs_hat:  {coeffs_hat.round(2)}')
print(f'  - coeffs_true: {coeffs_true.round(2)}')
print(f'  - max_diff: {max_diff:.4f}')
if max_diff < 0.1:
    print('  - Test SUCESS! (max_diff < 0.1)')
else:
    print('  - Test FAILED !!!')

  - Run Granger Busca...
  - coeffs_hat:  [0.31 0.28 1.34 1.36 1.34 1.36 0.78 0.22 0.22 0.78]
  - coeffs_true: [0.40 0.10 1.10 1.00 1.20 1.20 0.70 0.30 0.00 1.00]
  - max_diff: 0.3593
  - Test FAILED !!!


In [8]:
beta_hat

array([[1.34, 1.36],
       [1.34, 1.36]])

In [11]:
np.ones((dim, dim)) * (granger_model.beta_ + 1)

array([[1.34, 1.36],
       [1.34, 1.36]])